# Markdown cell


In [7]:
!pip install --quiet streamlit google-cloud-aiplatform langchain-google-vertexai google-cloud-vertexai

In [ ]:
%%bash
gcloud compute firewall-rules create allow-imc-streamlit \
    --direction=INGRESS \
    --priority=1000 \
    --network=default \
    --action=ALLOW \
    --rules=tcp:8502 \
    --source-ranges=0.0.0.0/0 2> /dev/null

gcloud compute firewall-rules list --filter name='allow-imc-streamlit' 2> /dev/null

In [8]:
%%writefile swm-chat.py
import streamlit as st
import google.auth;
import vertexai
from vertexai.generative_models import GenerativeModel, ChatSession, GenerationConfig, Tool, Part, FunctionDeclaration

credentials, project_id = google.auth.default()
vertexai.init(project=project_id, location="us-central1")
foundation_models = ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro"]
contexto = """
Eres un asistente virtual que ayuda a nadadores a obtener información sobre el mundo de la natación,
como por ejemplo extraer datos de las normativas de las competiciones, records y marcas personales de otros nadadores o
noticias generales relacionadas. Es importante solo responder a consultas relacionadas con la natación.
"""

model_name = st.selectbox("Models:", foundation_models)
#el modelo no puede cambiar en ejecución averiguar como o averiguar si de verdad es así


def nivel_natacion_libres(metros: int, minutos: int) -> str:
    if (metros == 50):
            if minutos > 1:
                return "nivel municipal"
            elif minutos < 1:
                return "nivel nacional"
            else:
                return "nivel regional"
    else:
        return "muchos metros"

herramientas_mapeo =  {"nivel_natacion_libres": nivel_natacion_libres}

herramientas_agente = Tool(
    function_declarations =[
        FunctionDeclaration(name = "nivel_natacion_libres",
                                                parameters={
                                                    "type": "object",
                                                    "properties": {
                                                        "metros": {"type": "integer", "description": "quantity of meters swam"},
                                                        "minutos": {"type": "integer", "description": "ammount of time swam"},
                                                    },
                                                    "required": ["metros","minutos"],
                                                },)
                            ]
)

model = GenerativeModel(model_name,
                        system_instruction = contexto,
                        generation_config = GenerationConfig(
                            temperature = 0.0,
                            max_output_tokens = 100
                        ),
                        tools = [herramientas_agente]
)


st.set_page_config(page_title="Aplicación Streamlit", page_icon="⚙️", layout="centered")

st.title("Chatbot")
st.subheader(f"Conectado a {project_id} con modelo:{model_name}")


if "chat" not in st.session_state and model:
    st.session_state.chat = model.start_chat()

if model:
    prompt = st.text_area("Introduce tu prompt:")
    if st.button("Generar Respuesta"):
        # --- Mostrar historial previo
        if prompt.strip():
            with st.chat_message("user"):
                st.write(prompt)
            # ----------------------------------
            #  Lógica del Agente
            # ----------------------------------
            with st.chat_message("assistant"):
                with st.spinner("El agente está pensando..."):
                    try:
                        # --- Envío del mensaje al modelo
                        response = st.session_state.chat.send_message(prompt)
                        # --- Comprobación de si el Agente necesita usar una herramienta (Function Calling)
                        part = response.candidates[0].content.parts[0]
                        if part.function_call:
                            st.info(f"🛠️ El agente está usando la herramienta: `{part.function_call.name}`")
                            # --- Extraer nombre de la función y argumentos
                            fn_name = part.function_call.name
                            fn_args = {key: val for key, val in part.function_call.args.items()}
                            # --- Ejecutar la función real de Python
                            if fn_name in herramientas_mapeo:
                                api_response = herramientas_mapeo[fn_name](**fn_args)
                                # --- Devolver el resultado de la función al modelo para que genere la respuesta final
                                response = st.session_state.chat.send_message(
                                    Part.from_function_response(
                                        name=fn_name,
                                        response={
                                            "content": api_response,
                                        },
                                    )
                                )
                        # --- Mostrar la respuesta final (texto natural)
                        st.write(response.text)
                    except Exception as e:
                        st.error(f"Error en la lógica del agente: {str(e)}")
else:
    st.error("Error de inicialización. Revisa logs.")


#prompt = st.chat_input("Say something")
#if prompt:
#    response = model.generate_content(prompt)
#    st.write(f"User has sent the following prompt: {response.text}")

Overwriting SWMChat.py


In [4]:
!streamlit run SWMChat.py --server.port 8502

^C
